In [1]:
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
quandl.ApiConfig.api_key = 'iB1vwTHbXzBdG755yy2W'

In [3]:
## returns direction of a time sorted df from earliest time -> more recent time, start date and end date.
def get_df_date_info(series):
    d1 = series.iloc[0].name
    d2 = series.iloc[-1].name
    if d1 < d2:
        #low -> high
        return [1, d1, d2]
    if d2 < d1:
        #high -> low
        return [-1, d2, d1]

class Macro_Series:
    def __init__(self,name,dataframe):
        self.name = name
        self.df = dataframe
        
        time_data = get_df_date_info(dataframe)
        self.direction = time_data[0]
        self.start_date = time_data[1]
        self.end_date = time_data[2]
        print(self.name)

In [4]:
class Stock_Macro:
    def __init__(self,stock_code,dateframe):
        self.stock_code = stock_code
        self.df = dataframe
        
        time_data = get_df_date_info(dataframe)
        self.direction = time_data[0]
        self.start_date = time_data[1]
        self.end_date = time_data[2]

In [5]:
def get_macro_data():
    macro_data = {}
    macro_data['us_tr_bill_3m'] = Macro_Series( 'us_tr_bill_3m',quandl.get('FRED/TB3MS',column_index='1') )
    macro_data['us_gdp_potential'] = Macro_Series( 'us_gdp_potential',quandl.get('FRED/GDPPOT', column_index='1') )
    macro_data['us_nat_unemployment'] = Macro_Series( 'us_nat_unemployment',quandl.get('FRED/NROUST',column_index='1') )

    pred_gov_net_lending_keys = {'FRED/GGNLBPDEA188N':'Germany','FRED/GGNLBPFRA188N':'France','FRED/GGNLBPJPA188N':'Japan','FRED/GGNLBPCAA188N':'Canada',
                                 'FRED/GGNLBPCNA188N':'China','FRED/GGNLBPRUA188N':'Russia','FRED/GGNLBAGBA188N':'UK','FRED/GGNLBPEZA188N':'EU',
                                 'FRED/GGNLBPUSA188N':'US','FRED/GGNLBAINA188N':'India'}

    pred_gov_net_lending = []
    for i in pred_gov_net_lending_keys:
        name = pred_gov_net_lending_keys[i] + '_pred_gov_net_lending'
        pred_gov_net_lending.append( Macro_Series(name,quandl.get(i,column_index='1') ))
    macro_data['pred_gov_net_lending'] = pred_gov_net_lending
    
    macro_data['fed_total_public_debt'] = Macro_Series( 'fed_total_public_debt', quandl.get('FRED/GFDEBTN',column_index='1') )

    gov_gross_debt_keys = {'FRED/GGGDTADEA188N':'Germany','FRED/GGGDTAFRA188N':'France','FRED/GGGDTAJPA188N':'Japan','FRED/GGGDTACAA188N':'Canada',
                           'FRED/GGGDTACNA188N':'China','FRED/GGGDTARUA188N':'Russia','FRED/GGGDTAGBA188N':'UK','FRED/GGGDTAEZA188N':'EU',
                           'FRED/GGGDTAUSA188N':'US','FRED/GGGDTAINA188N':'India',}
    gov_gross_debt = []
    for i in gov_gross_debt_keys:
        name = gov_gross_debt_keys[i] + '_gov_gross_debt'
        gov_gross_debt.append( Macro_Series(name,quandl.get(i,column_index='1')) )
    macro_data['gov_gross_debt'] = gov_gross_debt
    
    macro_data['crude_oil_historical'] = Macro_Series('crude_oil_historical', quandl.get('BP/CRUDE_OIL_PRICES',column_index='1') )
    
    macro_data['crude_oil_opec'] = Macro_Series('crude_oil_opec',quandl.get('OPEC/ORB',column_index='1'))
    
    macro_data['gold_price'] = Macro_Series('gold_price', quandl.get('LBMA/GOLD',column_index='1'))
    return macro_data

In [6]:
mc_data = get_macro_data()

us_tr_bill_3m
us_gdp_potential
us_nat_unemployment
Germany_pred_gov_net_lending
France_pred_gov_net_lending
Japan_pred_gov_net_lending
Canada_pred_gov_net_lending
China_pred_gov_net_lending
Russia_pred_gov_net_lending
UK_pred_gov_net_lending
EU_pred_gov_net_lending
US_pred_gov_net_lending
India_pred_gov_net_lending
fed_total_public_debt
Germany_gov_gross_debt
France_gov_gross_debt
Japan_gov_gross_debt
Canada_gov_gross_debt
China_gov_gross_debt
Russia_gov_gross_debt
UK_gov_gross_debt
EU_gov_gross_debt
US_gov_gross_debt
India_gov_gross_debt
crude_oil_historical
crude_oil_opec
gold_price


In [7]:
def analyse_macro_data(macro_data,stock_data):
    print("Length:",len(macro_data))
    for i in macro_data:
        print('-------------')
        print(i)
        print(type(macro_data[i]))
        macro_type_error = str('Value inputted:\n\n' + str(macro_data[i]) + '\n\nIs not of type list or class Macro_Series. It is of type ' + str(type(macro_data[i])))
        if type(macro_data[i]) is Macro_Series:
            ## Macro Series analysis - global factor
            print('Macro_Series analysis')
            
            
            
        elif type(macro_data[i]) is list:
            for j in macro_data[i]:
                print(j.name)
                if type(j) is Macro_Series:
                    ## Macro Series analysis - per country
                    print('Macro_Series analysis - per country')
                    
                    
                    
                else:
                    raise ValueError(macro_type_error)
        else:
            raise ValueError(macro_type_error)
    return

In [8]:
def read_format_stock(path):
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df['date'])
    df['1. open'] = df['1. open'].astype(float)
    df['2. high'] = df['2. high'].astype(float)
    df['3. low'] = df['3. low'].astype(float)
    df['4. close'] = df['4. close'].astype(float)
    df['5. volume'] = df['5. volume'].astype(float)
    return df

In [9]:
googl_path = os.path.join(os.getcwd(),"..","..","Data","GOOGL2.csv")
googl_df = read_format_stock(googl_path)
googl_df.head()

,date,1. open,2. high,3. low,4. close,5. volume
0,2004-08-19,100.01,104.06,95.96,100.335,44659000.0
1,2004-08-20,101.01,109.08,100.50,108.310,22834300.0
2,2004-08-23,110.76,113.48,109.05,109.400,18256100.0
3,2004-08-24,111.24,111.60,103.57,104.870,15247300.0
4,2004-08-25,104.76,108.00,103.88,106.000,9188600.0


In [10]:
analyse_macro_data(mc_data,googl_df)

Length: 9
-------------
us_tr_bill_3m
<class '__main__.Macro_Series'>
Macro_Series analysis
-------------
us_gdp_potential
<class '__main__.Macro_Series'>
Macro_Series analysis
-------------
us_nat_unemployment
<class '__main__.Macro_Series'>
Macro_Series analysis
-------------
pred_gov_net_lending
<class 'list'>
Germany_pred_gov_net_lending
Macro_Series analysis - per country
France_pred_gov_net_lending
Macro_Series analysis - per country
Japan_pred_gov_net_lending
Macro_Series analysis - per country
Canada_pred_gov_net_lending
Macro_Series analysis - per country
China_pred_gov_net_lending
Macro_Series analysis - per country
Russia_pred_gov_net_lending
Macro_Series analysis - per country
UK_pred_gov_net_lending
Macro_Series analysis - per country
EU_pred_gov_net_lending
Macro_Series analysis - per country
US_pred_gov_net_lending
Macro_Series analysis - per country
India_pred_gov_net_lending
Macro_Series analysis - per country
-------------
fed_total_public_debt
<class '__main__.Macro_